Requirements - A checklist to remind us if we tick all the boxes
---------------------
- [ ] Make a nice plot to select a time/ period of interest
- [ ] Run WIT on a per-pixel basis
- [ ] Return Water/Wet/FC percentage per pixel
- [ ] Plot and output WIT spatially, with FC percentage represented as an alpha % for the colour
- [ ] Output the results as a ArcGIS-compliant Geotiff (uint8), with the shapefile name and the date in the filename

Functions or functionalities
---------
- bokeh wit plot: to do a stack plot of wit data with bokeh
    - input: DataFrame
    - output: stack plot of wit data
- load_wit_data(**kwargs)
    - input: csv file or poly_id in database
    - output: DataFrame
- load_wofs_fc(query)
    - input: a query dictionary with time and geometry
    - output: an xarray with water/wet/FC percentage
- plot_spatial_wit(input_pixels_array)
    - input: an xarray with water/wet/FC percentage
    - output: 2 dimensional plot of input
- write_geotiff(input_pixels_array, file_name)
    - input: an xarray with water/wet/FC percentage
            a string as file anme
    - output: a geotiff file with input file name
    
Coding/writing style requirements
-------------------------
- Always break up a sentence if it's too long, slide showing up is a good indicator
- Merge all the cells without essential output unless requied not so, e.g, explanation proceeding a functionality
- Use `_LOG.debug` provided insted of `print`
- Follow the comments in cells to fill in code rather than randomly dump
- Keep all the variables humanly readable, a, b, c or aa, bb, cc etc. simple letters or their permutation are forbidden to be used globally
- Something else I haven't come up with yet

Does the front end plumbing that you're designing for WIT spatial encompass multiple potential sources of WIT?
- Retrieve data from DB
- User uploads WIT CSV file (ie if WA wanted to upload one of the WITs we've just shipped them)
- Retrieve pre-calculated WIT from S3 bucket

In [ ]:
# import all the necessary packages in this cell
# check if anything you want is here already with different abbr. with Ctrl+F (maybe there is a better way?
# but you got the idea)
import pandas as pd
import numpy as np
from datacube import Datacube
from datetime import datetime, timedelta
import matplotlib.dates as mdates

from bokeh.io import curdoc, output_notebook, show
from bokeh.layouts import layout, column, row, WidgetBox, gridplot
from bokeh.models import CheckboxGroup, Select,  CategoricalColorMapper, ColumnDataSource,HoverTool, Label, SingleIntervalTicker, Slider, DatetimeTickFormatter, YearsTicker, Legend, TapTool, CustomJS, LegendItem, field, Range1d
from bokeh.models.formatters import DatetimeTickFormatter
from bokeh.models.glyphs import Text
from bokeh.models.tickers import DatetimeTicker
from bokeh.palettes import viridis, brewer
from bokeh.plotting import figure, output_file, show
from bokeh.transform import factor_cmap, LinearColorMapper
from bokeh.events import DoubleTap
import os, sys, urllib, logging
import seaborn as sns



output_notebook()
_LOG = logging.getLogger('spatial_wit')
stdout_hdlr = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter('[%(asctime)s.%(msecs)03d - %(levelname)s] %(message)s')
stdout_hdlr.setFormatter(formatter)
_LOG.addHandler(stdout_hdlr)
_LOG.setLevel(logging.DEBUG)

In [ ]:
# put global variables in this cell
shapefile = '/g/data1a/r78/DEA_Wetlands/shapefiles/ramsar_wetlands_3577_20190403.shp'

Here is the start of all functions
----------------------------------------------
Note: Put each function in a cell

In [ ]:
def load_wit_data(url='https://dea-public-data-dev.s3-ap-southeast-2.amazonaws.com/Wetlands_Insight_Tool/WIT_v3/Kerang%20Wetlands_Hird%20Swamp_VIC_17.csv'):
    '''eventually we want a function that will load data from a csv, 
    or retrieve the data from the DB, or get the data from an s3 bucket
    then reformat dataframe to get nice consistent format
    Parameters:
    -----------
    shapefile specific parameters to parse filenames when retrieving data?
    url'''
    #currently set to get a single WIT csv from the s3 bucket to demonstrate the plotting code
    wit_df = pd.read_csv(url, index_col =0, infer_datetime_format=True)
    
    #give the index a name that reflects that it is time, measured in UTC not AEDT/AEST
    wit_df.index.names = ['utc_time']

    #format the index of the dataframe as a date, not as a string
    wit_df.index = pd.to_datetime(wit_df.index)
    #Rename the columns so they are easier to understand and plot
    wit_df = wit_df.rename(columns={"WATER" : "water", 
                            "WET" : "wet",
                           "PV" : "green",
                           "NPV" : "dry",
                           "BS" : "bare"}) 
    #converting to percentages to make plotting easier
    #first convert if not already a percentage
    if wit_df.max().max() <=1.0:
        wit_df = wit_df*100
    #WITdata.head()
    return wit_df

In [ ]:
def bokeh_WIT_plot(WITdata, polyName='provided polygon'):
    '''
    last modified: May 2020
    
    Parameters
    ----------
    WITdata : xarray data array produced by load_wit_data function
    polyName : string
               A name for the polygon to identify the plot, optional. Defaults to 'provided polygon' 
                   
    Returns
    -------
    A bokeh stack plot of the contents of the vector file in water, wet, green, dry and bare. Plot can be zoomed in to select a date. 
    '''
    
    #set up color palate for bokeh WIT plot
    pal = [sns.xkcd_rgb["cobalt blue"],
           sns.xkcd_rgb["neon blue"],
           sns.xkcd_rgb["grass"],
           sns.xkcd_rgb["beige"],
           sns.xkcd_rgb["brown"]]  

    #these are tools we want to use in the plot
    TOOLS = ["pan, wheel_zoom, box_zoom, reset, tap, save"]

    #lets put a title on the plot
    title =f'Percentage of area dominated by WOfS, Wetness, Fractional Cover for {polyName}'    

    #set up the x axis to recognise date and time. Note that you will only see the days when you zoom in.
    p =figure(plot_width=1200, 
              plot_height = 400, 
              x_axis_type='datetime',
             title=title, tools=TOOLS)
    p.sizing_mode = "scale_width"

    #align the title in the centre
    p.title.align= "center"
    p.title.text_font_size="12pt"

    #label axes
    p.yaxis.axis_label=("percentage of polygon classified as type")
    p.yaxis.axis_label_text_font_size="8pt"

    #we need screen units to put the attribution label under the plot. Don't ask why.
    label_opts = dict(
        x=0, 
        y=0,
        x_units='screen', 
        y_units='screen',
        text_font_style="italic", 
        text_font_size="8.5pt")
    
    #underplot context
    msg1 = 'The Fractional Cover algorithm developed by the Joint Remote Sensing Research Program\n\
    and the Water Observations from Space algorithm developed by Geoscience Australia are used in the production of this data'
    caption1 = Label(text=msg1, **label_opts)

    p.add_layout(caption1, 'below')

    p.xaxis.formatter=DatetimeTickFormatter(years =["%Y"], months=["%m/%Y"] ,days=["%d/%m/%Y"])
    p.xaxis.major_label_orientation = 45

    #create the actual stack plot using data from the pandas dataframe 
    p.varea_stack(['water', 
                  'wet',
                  'green',
                  'dry',
                  'bare'], x= 'utc_time', color=pal, fill_alpha=0.7, source = WITdata, 
                  legend_label=["water","wet","green","dry","bare"], muted_color="grey", muted_alpha=0.2)


    #set the new WIT graph ranges.
    left, right, bottom, top = WITdata.index[0], WITdata.index[-1], 0, 100 #set 
    p.x_range=Range1d(left, right)
    p.y_range=Range1d(bottom, top)
    p.xaxis.bounds=(left,right)
    p.yaxis.bounds=(bottom,top)

    #now we want to overplot the data on the plot
    #create rectangle borders for no-data times (SLC-off only)
    LS5_8_gap_start = datetime(2011,11,1)
    LS5_8_gap_end = datetime(2013,4,1)

    #plot our dead satellite rectangle
    p.hbar(y=50, 
           height=100,
           left=LS5_8_gap_start, 
           right=LS5_8_gap_end, 
           color="white", 
           alpha=0.5, 
           hatch_color="white", 
           hatch_pattern='/',
           hatch_alpha=0.6,
           line_color="white",
           line_width =2,
           line_alpha=0.6)

    p.legend
    p.legend.location="bottom_left"
    p.legend.click_policy="mute"
    p.legend.background_fill_alpha=0.5
    p.legend.border_line_alpha=0.5
    p.legend.label_text_font_size="9pt"

    #display the plot    
    show(p)   

    return #p


Here is the start of all non function functionalities
------------------------------------------------------------------------
Note: Keep a single functionality in a SINGLE cell; DONOT make functionalities tangled 

In [ ]:
#load some WIT data
WITdata =load_wit_data()

In [ ]:
#put in the name of your polygon here (or extract it from something else?)

In [ ]:
polyName = "Hird Swamp" #change this as appropriate

In [ ]:
bokeh_WIT_plot(WITdata, polyName)